<p align = "center">
    <img src="img/cinta.png" />
</p>

<p align = "center">
    <img src="img/logo.png" />
</p>


# **Maestría en Ciencia de Datos**
## Temas Selectos de Modelado. Primavera 2022.

### **Equipo 1** 

- Nyrma Paulina Hernández Trejo
- Aide Jazmín González Cruz
- Joel Jaramillo Pacheco
- Jesús Enrique Miranda Blanco

<br>

<div align="center"><h1 style="font-weight:bold; font-size: 40px">Práctica 2</h1></div>


# **Segunda parte**

Se muestra la ejecución del paquete en el que se muestra el mejoramiento realizado. Para ello se cargan las librerías necesarias:

In [1]:
from IPython.display import display, HTML, Image
from datetime import date
import pandas as pd
import random
import numpy as np 
import networkx as nx
import time

Y se muestran las características de la máquina donde se corrió el código:

In [9]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   36 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           58
Model name:                      Intel(R) Core(TM) i5-3210M CPU @ 2.50GHz
Stepping:                        9
CPU MHz:                         2194.284
CPU max MHz:                     3100.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4988.64
Virtualization:                  VT-x
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        512 KiB
L3 cache:                       

In [ ]:
%%bash
sudo lshw -C memory

In [11]:
%%bash
uname -ar #r for kernel, a for all

Linux 5a4ea55e41da 5.13.0-41-generic #46~20.04.1-Ubuntu SMP Wed Apr 20 13:16:21 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


- **Código original**

In [2]:
def get_data(data_dir, date=date.today().strftime("%Y-%m-%d"), tam_data=100, price='Open'):
    """
    Obtiene datos de un directorio de una fecha dada
    param:
    data_dir: directorio de datos
    date: fecha a analizar, por default toma la fecha actual si no se especifica
    tam_data: analiza cierto tamaño del dataset de forma aleatoria, por default son 100 monedas a analizar
    price: selecciona con que precio se hará el ejercicio(High,Low,Open,Close), por default es Open
    return:
    dataframe: con datos especificados
    """
    df = pd.read_csv(data_dir)
    df_date = df[df['Date']==date]
    df_date = df_date[df_date[price] > 0]
    
    if(tam_data == None):
        df_random = df_date
    else:
        df_random = df_date.sample(n = tam_data)
    
    
    df_random = df_random.reset_index()
    data = df_random[["ticker", price]]
    data.columns = ['Símbolo', 'Precio']
    
    return data    

def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
    c1.drop(columns=['Precio'],inplace=True)
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1
    return c1

def log_transformed_rep(data):
    """
    Log-Transformed Representations
    param:
        dataframe
    return:
        dataframe
    """
    
    df_ln = np.round(-np.log(data),2)
    return df_ln

def create_grap(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:  
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

- **Código perfilado**

In [3]:
def exchange_rate_matrix2(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
        
    c1.drop(columns=['Precio'],inplace=True)
    
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1    
    return c1

def create_grap2(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix2(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

def bf_negative_cycle2(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

## **Tiempo**

Se muestra la corrida del paquete anterior vs el perfilado con *time* y *timeit*

### ***Time***

- Código original proceso completo

In [6]:
start_time_c = time.time()
df = get_data('data/historical_data.csv', '2022-05-12')
G = create_grap(df)
start_time_m = time.time()
bf_negative_cycle(G)
end_time = time.time()
secs_c = end_time-start_time_c
secs_m = end_time-start_time_m
print("Arbitrage Identification Cycle in Crypto Trading tomó: ",secs_c,"segundos")
print("Bellman ford tomó: ",secs_m,"segundos")

Arbitrage Identification Cycle in Crypto Trading tomó:  4.627994060516357 segundos
Bellman ford tomó:  3.1854867935180664 segundos


- Código perfilado proceso completo

In [8]:
start_time_c = time.time()
df = get_data('data/historical_data.csv', '2022-05-12')
G = create_grap2(df)
start_time_m = time.time()
bf_negative_cycle2(G)
end_time = time.time()
secs_c = end_time-start_time_c
secs_m = end_time-start_time_m
print("Arbitrage Identification Cycle in Crypto Trading tomó: ",secs_c,"segundos")
print("Bellman ford tomó: ",secs_m,"segundos")

Arbitrage Identification Cycle in Crypto Trading tomó:  3.5411875247955322 segundos
Bellman ford tomó:  2.1046359539031982 segundos


### ***Timeit***

- Bellman ford trabajado en prácticas anteriores

In [13]:
%timeit bf_negative_cycle(G)

2.69 s ± 73.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- Bellman ford perfilado

In [14]:
%timeit bf_negative_cycle2(G)

2.03 s ± 93.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ***cProfile***

## **Memoria**

Se muestra la corridas con los paquetes para ver el mejoramiento usando *memory_profiler*